# Importing the Dependencies

In [1]:
import os
import tensorflow
from tensorflow import keras
from keras.preprocessing import image
import cv2 
import random
import numpy as np

In [2]:
categories = ['with_mask', 'without_mask']

### For Train Data

In [3]:
data = []
for label, category in enumerate(categories):
    path = os.path.join('Train', category)

    if not os.path.exists(path):  
        print(f"Warning: Directory {path} does not exist.")
        continue

    for file in os.listdir(path):
        img_path = os.path.join(path, file)

        # Skip non-image files
        if not file.lower().endswith(('.png', '.jpg', '.jpeg')):
            print(f"Skipping non-image file: {file}")
            continue

        img = cv2.imread(img_path)

        if img is None:  # Skip unreadable images
            print(f"Warning: Could not read image {img_path}. Skipping.")
            continue

        # Now it's safe to resize
        img = cv2.resize(img, (224, 224))  # Resize image
        img = img / 255.0  # Normalize pixel values

        data.append([img, label])  # Store as tuple (image, label)


Skipping non-image file: .DS_Store
Skipping non-image file: .DS_Store


In [4]:
print(f"Total images loaded: {len(data)}")

Total images loaded: 6005


In [5]:
random.shuffle(data)

In [6]:
X = []
y = []

for features, label in data:
    X.append(features)
    y.append(label)

In [7]:
print("Lenght of X: ",len(X))
print("Lenght of y: ",len(y))

Lenght of X:  6005
Lenght of y:  6005


In [8]:
X = np.array(X)
y = np.array(y)

In [9]:
print(X.shape)
print(y.shape)

(6005, 224, 224, 3)
(6005,)


In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 2)

In [11]:
print("Shape of X_train : ", X_train.shape)
print("Shape of X_test : ", X_test.shape)
print("Shape of y_train : ", y_train.shape)
print("Shape of y_test : ", y_test.shape)

Shape of X_train :  (4804, 224, 224, 3)
Shape of X_test :  (1201, 224, 224, 3)
Shape of y_train :  (4804,)
Shape of y_test :  (1201,)


In [12]:
from keras.applications.vgg16 import VGG16

In [13]:
vgg = VGG16()

In [14]:
from keras import Sequential

In [15]:
model = Sequential()

In [16]:
for layer in vgg.layers[:-1]:
    model.add(layer)

In [17]:
for layer in model.layers:
    layer.trainable = False

In [18]:
from keras.layers import Dense

In [19]:
model.add(Dense(1, activation = "sigmoid"))

In [20]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc1 (Dense)                     │ (None, 4096)           │   102,764,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc2 (Dense)                     │ (None, 4096)           │    16,781,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │         4,097 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 134,264,641 (512.18 MB)

 Trainable params: 4,097 (16.00 KB)

 Non-trainable params: 134,260,544 (512.16 MB)

In [21]:
model.compile(optimizer = "Adam", loss = "binary_crossentropy", metrics = ["accuracy"])

In [22]:
model.fit(X_train, y_train, validation_data = (X_test, y_test), epochs = 5)

Epoch 1/5
151/151 ━━━━━━━━━━━━━━━━━━━━ 961s 6s/step - accuracy: 0.7090 - loss: 0.5786 - val_accuracy: 0.8984 - val_loss: 0.3119
Epoch 2/5
151/151 ━━━━━━━━━━━━━━━━━━━━ 766s 5s/step - accuracy: 0.8947 - loss: 0.2994 - val_accuracy: 0.9259 - val_loss: 0.2377
Epoch 3/5
151/151 ━━━━━━━━━━━━━━━━━━━━ 796s 5s/step - accuracy: 0.9262 - loss: 0.2329 - val_accuracy: 0.9342 - val_loss: 0.2051
Epoch 4/5
151/151 ━━━━━━━━━━━━━━━━━━━━ 821s 5s/step - accuracy: 0.9317 - loss: 0.2024 - val_accuracy: 0.9259 - val_loss: 0.1931
Epoch 5/5
151/151 ━━━━━━━━━━━━━━━━━━━━ 788s 5s/step - accuracy: 0.9256 - loss: 0.1959 - val_accuracy: 0.9384 - val_loss: 0.1719


In [23]:
def detect_face_mask(img):
    y_pred = model.predict(img.reshape(1,224,224,3))

    return y_pred[0][0]

In [24]:
def draw_label(img, text,pos,bg_color):

    text_size = cv2.getTextSize(text,cv2.FONT_HERSHEY_SIMPLEX,1,cv2.FILLED)

    end_x = pos[0] + text_size[0][0] + 2
    end_y = pos[1] + text_size[0][1] - 2

    cv2.rectangle(img, pos, (end_x, end_y), bg_color, cv2.FILLED)
    cv2.putText(img, text, pos, cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0),1,cv2.LINE_AA)

In [25]:
haar = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

In [30]:
def detect_face(img):
    coods = haar.detectMultiScale(img)

    return coods

In [31]:
cap = cv2.VideoCapture(0)

In [ ]:
while True:
    ret, frame = cap.read()

    # Call the detection method

    img = cv2.resize(frame, (224,224))

    y_pred = detect_face_mask(img)

    if y_pred == 0:
        draw_label(frame, "Mask", (30,30), (0,255,0))
    
    else:
        draw_label(frame, "No Mask", (30,30), (0,0,255))

    coords = detect_face(frame) 
    for x, y, w, h in coords:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0),3)

    # Wait for 'x' key press to exit
    cv2.imshow("Window", frame)

    if cv2.waitKey(1) & 0xFF == ord('x'):
        break

cap.release()
cv2.destroyAllWindows()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 263ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 169ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 271ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10